In [2]:
from bs4 import BeautifulSoup
import csv
import requests
import pandas as pd

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

In [3]:
product_name = '김치'
# https://www.coupang.com/np/search?component=&q=김치&channel=user
url = f"https://www.coupang.com/np/search?component=&q={product_name}&channel=user"

In [4]:
# get html
response = requests.get(url, headers = headers)
response

<Response [200]>

In [5]:
# BeautifulSoup을 활용하여 데이터 파싱
soup = BeautifulSoup(response.text, "html.parser")
# print(soup)

In [6]:
#상품명
# <div class="name">비비고 썰은배추김치, 1.8kg, 1개</div>
names = soup.findAll('div', {'class':'name'})

# for item in names :
#     print(item.text)

In [7]:

# <div class="price">  <span class="price-info">  <span class="instant-discount-rate">6%</span>   <del class="base-price">14,900</del>  </span> <em class="sale "> <strong class="price-value">13,890</strong>원   <span class="badge rocket"> <img src="//image6.coupangcdn.com/image/badges/falcon/v1/web/rocket-fresh@2x.png" height="16" alt="로켓배송">  </span>  </em>   <span class="unit-price">(<em>100</em>g당 <em>772</em>원) </span>  </div>
# <strong class="price-value">13,890</strong>원 
prices = soup.findAll('strong', {'class':'price-value'})

# for item in prices :
#     print(item.text)

In [8]:
name_list = []
price_list = []

In [9]:
# 광고 데이터(i=0일때)는 가격이 나오지 않으므로 리스트 생성 시, 제외
i = 0
for item in names :
    if(i > 0) : 
        name_list.append(item.text)        
    i = i + 1
    
for item in prices :
    price_list.append(item.text)

In [10]:
result_file_name = "coupang_kimch.csv"
for k in range(len(price_list)) :
#     print("*상품명 : ", name_list[k+1], ",    *가격 : ",price_list[k])
    result = pd.DataFrame(data={'상품명' : name_list, '가격' : price_list})
    result.to_csv(result_file_name, header = False, index = False, encoding='utf-8-sig')

In [11]:
## (실습1) 쿠팡 케이스별로 크롤링 하는 경우
product_name = '종가집 김치'
page_no=1
min_price=20000
max_price=40000

result_file_name ='쿠팡_result_2'

url = f"https://www.coupang.com/np/search?component=&q={product_name}&channel=user&page={page_no}&minPrice={min_price}&maxPrice={max_price}"

## 연습문제

- 쿠팡 웹 크롤링한 데이터를 csv 형태로 저장하기

In [7]:
df = pd.read_csv('coupang_kimch.csv')
#df = pd.read_csv('coupang_kimch.csv', header=None, names=['prouct', 'price'])
df.head()

,"친정김치 배추김치 5kg 10kg, 1개","19,900"
0,"비비고 썰은배추김치, 1.8kg, 1개","14,110"
1,"종가집 포기김치, 4kg, 1개","24,850"
2,"곰곰 국내산 포기 김치, 4kg, 1개","19,480"
3,"맛군 30년 전통! 국내산 전라도 전통 묵은지 묵은김치, 1팩, 3kg","13,900"
4,"국내산 재료 100% 천하일미 송가네 여수 돌산 갓김치 4kg+갓물김치 1kg, 0...","39,800"


In [9]:
df = pd.read_csv('coupang_kimch.csv', header=None, names=['prouct', 'price'])
df.head()

,prouct,price
0,"친정김치 배추김치 5kg 10kg, 1개","19,900"
1,"비비고 썰은배추김치, 1.8kg, 1개","14,110"
2,"종가집 포기김치, 4kg, 1개","24,850"
3,"곰곰 국내산 포기 김치, 4kg, 1개","19,480"
4,"맛군 30년 전통! 국내산 전라도 전통 묵은지 묵은김치, 1팩, 3kg","13,900"


- 가격순으로 정렬하기

In [16]:
df['price'].sort_values()

9     10,110
13    10,400
19    11,310
8     11,800
34    11,900
27    12,000
4     13,900
1     14,110
10    14,290
6     14,490
28    16,060
20    17,000
11    17,060
26    18,900
3     19,480
29    19,900
0     19,900
14    20,630
32    21,500
17    21,900
2     24,850
16    27,900
23    38,010
30    38,300
5     39,800
18     7,520
7      7,990
15     8,400
21     8,720
25     8,850
12     8,900
24     9,000
22     9,220
35     9,280
31     9,860
33     9,900
Name: price, dtype: object

- 가장 높은 가격의 제품명(타이틀) 찾아보기

In [19]:
df[df['price'] == max(df['price'])]

,prouct,price
33,"장군차김치 모산 장군차 포기김치, 1.5kg, 1개","9,900"


- 가장 낮은 가격의 제품명(타이틀) 찾아보기

In [20]:
df[df['price'] == min(df['price'])]

,prouct,price
9,"곰곰 국내산 썰은 배추 김치, 1.8kg, 1개","10,110"


- 평균가격보다 높은 데이터 찾아보기

In [37]:
n_price = df.price

In [38]:
##(0702복습)지은이_pandas_기본 참조
def f(x):
    
    n_price = x.replace(',','')
    
    return int(n_price)

In [39]:
df['price']=df['price'].apply(f)

In [40]:
df.price.mean()

16198.333333333334

In [44]:
df[df['price']>df.price.mean()]

,prouct,price
0,"친정김치 배추김치 5kg 10kg, 1개",19900
2,"종가집 포기김치, 4kg, 1개",24850
3,"곰곰 국내산 포기 김치, 4kg, 1개",19480
5,"국내산 재료 100% 천하일미 송가네 여수 돌산 갓김치 4kg+갓물김치 1kg, 0...",39800
11,"종가집 썰어담은 맛김치, 1.7kg, 1봉",17060
14,"종가집 전라도 포기김치, 3.3kg, 1개",20630
16,"풀무원 톡톡 포기김치, 4kg, 1개",27900
17,"비비고 김장김치처럼 시원한 포기김치, 3kg, 1개",21900
20,"조은해남 전라도 배추김치 5kg 10kg HACCP인증 일반상품, 1박스",17000
23,"진경 배추김치, 10kg, 1봉",38010
